# Documention Agent

This notebook demonstrates a documention agent:
1. Video-to-protocol conversion using Vertex AI
2. With knowledge from few-shot-learning (video and protocol examples)

In [1]:
# %pip install google-cloud-storage
# %pip install --upgrade --user --quiet google-cloud-aiplatform

In [2]:
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path

import configparser
from IPython.display import Markdown

path_to_append = Path(Path.cwd()).parent / "proteomics_specialist"
sys.path.append(str(path_to_append))
import videoToProtocol as video_to_protocol

config = configparser.ConfigParser()
config.read("../secrets.ini")

['../secrets.ini']

In [3]:
import configparser
import vertexai

config = configparser.ConfigParser()
config.read("../secrets.ini")

PROJECT_ID = config["DEFAULT"]["PROJECT_ID"]
vertexai.init(project=PROJECT_ID, location="europe-west9") # europe-west9 is Paris

In [4]:
def upload_few_shot_examples(few_shot_example_folder, few_shot_examples, bucket, subfolder_in_bucket=None, custom_blob_name=None):
    uploaded_uris = {}
    
    for example in few_shot_examples:
        # Upload video file
        video_path = os.path.join(few_shot_example_folder, f"{example}.mp4")
        video_uri = video_to_protocol.upload_video_to_gcs(video_path, bucket, "fewShotExamples")
        
        # Upload protocol file
        protocol_path = os.path.join(few_shot_example_folder, f"{example}.md")
        protocol_uri = video_to_protocol.upload_video_to_gcs(protocol_path, bucket, "fewShotExamples")
        
        # Store the URIs in a dictionary
        uploaded_uris[example] = {
            "video": video_uri,
            "protocol": protocol_uri
        }
    
    return uploaded_uris

In [5]:
import os
from google.cloud import storage

os.environ["GOOGLE_CLOUD_PROJECT"] = config["DEFAULT"]["PROJECT_ID"]

# Initialize Cloud Storage client
storage_client = storage.Client()
bucket_name = "mannlab_videos"
bucket = storage_client.bucket(bucket_name)

In [6]:
# Upload few shot examples to Google Cloud Storage

subfolder_in_bucket = "fewShotExamples"

few_shot_example_folder = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/ready_examples/"

few_shot_examples = [
    "tims_calibration",
    "Refilling_tuneMix_at_timsTOFUltra",
    "Connect_ionOpticks_column",
    "Disconnect_IonOpticks_column_from_sample_line",
    "Placing_Evotips_on_Evosep"
]

uploaded_examples = upload_few_shot_examples(few_shot_example_folder, few_shot_examples, bucket, subfolder_in_bucket)
uploaded_examples

Uploaded to: gs://mannlab_videos/fewShotExamples/tims_calibration.mp4
Uploaded to: gs://mannlab_videos/fewShotExamples/tims_calibration.md
Uploaded to: gs://mannlab_videos/fewShotExamples/Refilling_tuneMix_at_timsTOFUltra.mp4
Uploaded to: gs://mannlab_videos/fewShotExamples/Refilling_tuneMix_at_timsTOFUltra.md
Uploaded to: gs://mannlab_videos/fewShotExamples/Connect_ionOpticks_column.mp4
Uploaded to: gs://mannlab_videos/fewShotExamples/Connect_ionOpticks_column.md
Uploaded to: gs://mannlab_videos/fewShotExamples/Disconnect_IonOpticks_column_from_sample_line.mp4
Uploaded to: gs://mannlab_videos/fewShotExamples/Disconnect_IonOpticks_column_from_sample_line.md
Uploaded to: gs://mannlab_videos/fewShotExamples/Placing_Evotips_on_Evosep.mp4
Uploaded to: gs://mannlab_videos/fewShotExamples/Placing_Evotips_on_Evosep.md


{'tims_calibration': {'video': 'gs://mannlab_videos/fewShotExamples/tims_calibration.mp4',
  'protocol': 'gs://mannlab_videos/fewShotExamples/tims_calibration.md'},
 'Refilling_tuneMix_at_timsTOFUltra': {'video': 'gs://mannlab_videos/fewShotExamples/Refilling_tuneMix_at_timsTOFUltra.mp4',
  'protocol': 'gs://mannlab_videos/fewShotExamples/Refilling_tuneMix_at_timsTOFUltra.md'},
 'Connect_ionOpticks_column': {'video': 'gs://mannlab_videos/fewShotExamples/Connect_ionOpticks_column.mp4',
  'protocol': 'gs://mannlab_videos/fewShotExamples/Connect_ionOpticks_column.md'},
 'Disconnect_IonOpticks_column_from_sample_line': {'video': 'gs://mannlab_videos/fewShotExamples/Disconnect_IonOpticks_column_from_sample_line.mp4',
  'protocol': 'gs://mannlab_videos/fewShotExamples/Disconnect_IonOpticks_column_from_sample_line.md'},
 'Placing_Evotips_on_Evosep': {'video': 'gs://mannlab_videos/fewShotExamples/Placing_Evotips_on_Evosep.mp4',
  'protocol': 'gs://mannlab_videos/fewShotExamples/Placing_Evotips

In [7]:
video_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/ready_examples/Disconnect_IonOpticks_column_from_sample_line.mp4"
video_uri_input = video_to_protocol.upload_video_to_gcs(video_path, bucket)

Uploaded to: gs://mannlab_videos/Disconnect_IonOpticks_column_from_sample_line.mp4


In [8]:
system_instructions = """
You are a highly observant research assistant in Professor Matthias Mann's proteomics and mass spectrometry laboratory. Your expertise lies in detailed documentation of experimental procedures.

Analyze the video and reconstruct a step-by-step protocol by focusing on the actions in the video. Focus on user interactions with equipment, devices, and software. The goal is a clear, concise, unambiguous protocol reproducible by someone with no prior knowledge. "Think aloud" as if you were the researcher in the video that describes their work. Describe what you see at every secound. 
Your task is to analyze the provided video and extract a detailed experimental protocol.
You always stick to the facts in the sources provided, and never make up new facts.
Take a deep breath and think step by step.
Answer direct.
"""

In [9]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part

inputs = []

# few shot examples
for i, uris in enumerate(uploaded_examples.values(), 1):
    video_expl_uri = uris['video']
    file_expl_uri = uris['protocol']
    
    inputs.extend([
        "Example Video {num}:",
        Part.from_uri(
            video_expl_uri, mime_type="video/mp4"
        ),
        "Example Protocol {num}:",
        Part.from_uri(
            file_expl_uri, mime_type="text/md"
        ),
    ])

# system prompts, and processed video
inputs.extend([
    system_instructions,
    "Video:",
    Part.from_uri(
        video_uri_input, mime_type="video/mp4"
    ),
    "Protocol:"
])

model = GenerativeModel("gemini-1.5-flash-002")
response = model.generate_content(
    inputs,
    generation_config={"temperature": 0} 
)
observation = response.text
print(response.usage_metadata)
Markdown(observation)

prompt_token_count: 134356
candidates_token_count: 614
total_token_count: 134970
prompt_tokens_details {
  modality: VIDEO
  token_count: 129675
}
prompt_tokens_details {
  modality: TEXT
  token_count: 4681
}
candidates_tokens_details {
  modality: TEXT
  token_count: 614
}



Here is a step-by-step protocol based on the provided video.  I will describe what is happening in each section of the video.

**Protocol: Connecting and Disconnecting Evosep Stage Tips**

**Abstract:** This protocol details the procedure for connecting and disconnecting Evosep stage tips (Evotips) on the Evosep One system.  Proper connection and disconnection are crucial for maintaining system integrity and preventing sample carryover.

**Materials:**

* Evosep One System
* Evotips (Evosep, 100 x 96 tips, cat. no. EV2018)
* Evotip Boxes (96-well format) filled with Buffer A (0.1% formic acid)

**Procedure:**

**Part 1: Connecting Evotips**

1. **(0:00-0:14):** The video begins with a view of the computer screen displaying the Evosep One software interface. The researcher then moves to the Evosep One system. Three empty Evotip boxes are visible in the rack system. The researcher picks up three new Evotip boxes from the bench.

2. **(0:14-0:15):** The researcher places the three Evotip boxes into the rack system on the Evosep One.

3. **(0:15-0:16):** The researcher checks the level of Buffer A in the Evotip boxes.

4. **(0:16-0:20):** The researcher picks up three new Evotip strips from the bench.

5. **(0:20-0:30):** The researcher carefully places the Evotips into the Evotip boxes, ensuring they are correctly oriented.

**Part 2: Disconnecting Evotips**

1. **(0:30-0:44):** The researcher approaches the Evosep One system. The researcher picks up the pliers.

2. **(0:44-0:50):** The researcher uses the pliers to carefully remove the Evotips from the Evotip boxes.

3. **(0:50-0:51):** The researcher places the used Evotips into a waste container.

4. **(0:51-1:00):** The researcher moves away from the Evosep One system.


**Expected Results:**

* Evotips are correctly placed in the Evotip boxes.
* Used Evotips are properly disposed of.

**Troubleshooting:**

* If Evotips are difficult to remove, use the pliers to gently loosen them.
* If Evotips are damaged or dry, discard them and replace them with new ones.

**Note:** The video does not show the entire process of running a sample on the Evosep One system.  This protocol only covers the connection and disconnection of the Evotips.  Additional steps would be required for a complete sample analysis protocol.